In [14]:
import os
import numpy as np
# import matplotlib.pyplot as plt
from datetime import datetime
import shutil


date_time_string = datetime.now().strftime('%Y.%m.%d %H-%M-%S')
InputPath = os.path.join('..','..', '..', 'inputNonEquilibriumRaw')
CopyPath = os.path.join('..','..', '..', 'copied', f'{date_time_string}_NonEquilibrium_Copied')
OutputPath= os.path.join('..','..','..', 'output', f'{date_time_string}_NonEquilibrium_Analyzed')
os.makedirs(OutputPath, exist_ok=True)
print(OutputPath)

def list_directories(path):
    return [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
    # return [os.path.join(path, name) for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]


directories = list_directories(InputPath)
print(len(directories))
print(directories)

# Open a text file in write mode
with open(os.path.join(OutputPath, 'SamplesInfo.txt'), 'w') as file:
    # Iterate over the list and write each item to the file
    for item in directories:
        file.write(item + '\n')  # Add a newline after each item

..\..\..\output\2024.09.02 10-13-18_NonEquilibrium_Analyzed
3
['2023.11.12_16-37-14_MacIsaac_Tr_L=32_thread=4_MCs=4.0e+00_ens=10000_Hist_T=0.850000', '2023.11.12_18-04-09_MacIsaac_Tr_L=64_thread=4_MCs=4.0e+00_ens=10000_Hist_T=0.850000', '2023.11.14_18-22-08_MacIsaac_Tr_L=128_thread=4_MCs=4.0e+00_ens=10000_Hist_T=0.850000']


In [15]:
# copy the directories into a tree structure based on their "L" and "T" values
import os
import re
import h5py
import shutil

for directory in directories:
    SubInputPath = os.path.join(InputPath , directory)
    L_value = re.search('L=(\d+)', directory).group(1)
    T_value = re.search('T=(\d+\.\d+)', directory).group(1)


    i = 1
    SubNewPath = os.path.join(CopyPath, L_value, T_value, f"{i}")
    while os.path.exists(SubNewPath):
        i += 1
        SubNewPath = os.path.join(CopyPath, L_value, T_value, f"{i}")

    os.makedirs(os.path.dirname(SubNewPath), exist_ok=True)
    shutil.copytree(SubInputPath, SubNewPath)

    # Now, store L_value and T_value in an HDF5 file
    hdf5_file_path = os.path.join(SubNewPath, 'metadata.h5')
    if not os.path.exists(hdf5_file_path):
        with h5py.File(hdf5_file_path, 'w') as hdf5_file:
            hdf5_file.create_dataset('SystemLength', data=int(L_value))
            hdf5_file.create_dataset('Temperature', data=float(T_value))

    

In [16]:
import os

def list_directories_at_level(root_directory, level):
    level_directories = []

    for root, dirs, files in os.walk(root_directory):
        # Check the level of the current directory
        current_level = root.count(os.sep) - root_directory.count(os.sep)

        if current_level == level:
            # Append subdirectories at the specified level to the list
            level_directories.extend(os.path.join(root, directory) for directory in dirs)

        # Avoid descending into subdirectories to keep it at the specified level
        if current_level >= level:
            dirs.clear()

    return level_directories

level_two_directories = list_directories_at_level(CopyPath, 1)

# Print the list of directories at level 1
print(level_two_directories)


['..\\..\\..\\copied\\2024.09.02 10-13-18_NonEquilibrium_Copied\\128\\0.850000', '..\\..\\..\\copied\\2024.09.02 10-13-18_NonEquilibrium_Copied\\32\\0.850000', '..\\..\\..\\copied\\2024.09.02 10-13-18_NonEquilibrium_Copied\\64\\0.850000']


In [17]:
# List to store the copied directories
CopiedDirectories = []

# Iterate over each source directory in level two directories
for source_directory in level_two_directories:
    # Split the source directory path
    split_directory = source_directory.split(os.path.sep)
    
    # Get the last two directories from the split directory path
    last_two_directories = split_directory[-2:]
    
    # Join the last two directories and add to the list
    CopiedDirectories.append(os.path.join(*last_two_directories))

# Print the copied directories
print(CopiedDirectories)

['128\\0.850000', '32\\0.850000', '64\\0.850000']


In [18]:
import os
import shutil

for CopiedDirectory in CopiedDirectories:
    SubOutputPath = os.path.join(OutputPath, CopiedDirectory)
    # Check if the target directory already exists, if not, create it
    if not os.path.exists(SubOutputPath):
        os.makedirs(SubOutputPath)
        print(f"Empty directory created at: {SubOutputPath}")
    else:
        print(f"Directory already exists: {SubOutputPath}")

Empty directory created at: ..\..\..\output\2024.09.02 10-13-18_NonEquilibrium_Analyzed\128\0.850000
Empty directory created at: ..\..\..\output\2024.09.02 10-13-18_NonEquilibrium_Analyzed\32\0.850000
Empty directory created at: ..\..\..\output\2024.09.02 10-13-18_NonEquilibrium_Analyzed\64\0.850000


In [19]:
import h5py
import numpy as np
import time
import re

Nbins = 5

start_time = time.time()
for CopiedDirectory in CopiedDirectories:
    print(f"directory = {CopiedDirectory}")
    # SubInputPath = os.path.join(CopyPath , directory, "1")
    SubOutputPath = os.path.join(OutputPath, CopiedDirectory)
    # os.makedirs(SubOutputPath, exist_ok=True)

    ResultsFiles = []
    metadatas = []
    # List of your HDF5 files
    counterDirectories = 1
    SubInputPath = os.path.join(CopyPath , CopiedDirectory, f"{counterDirectories}")
    while os.path.exists(SubInputPath):
        print(f"{SubInputPath} exists")
        ResultsFile = h5py.File(os.path.join(SubInputPath, 'Results.h5'), 'r')
        metadata = h5py.File(os.path.join(SubInputPath, 'metadata.h5'), 'r')
        ResultsFiles.append(ResultsFile)
        metadatas.append(metadata)
        counterDirectories += 1
        SubInputPath = os.path.join(CopyPath , CopiedDirectory, f"{counterDirectories}")


    # print(NensembleExpexted)
    Temporary_results = ResultsFiles[0][('/' + '0' + '/TimeSeries/Equilibration/Magnetization')]
    
    
    # Calculate the size of each subset
    m1timeSum = np.zeros((len(Temporary_results), Nbins))
    m2timeSum = np.zeros((len(Temporary_results), Nbins))
    m3timeSum = np.zeros((len(Temporary_results), Nbins))
    m4timeSum = np.zeros((len(Temporary_results), Nbins))
    E1timeSum = np.zeros((len(Temporary_results), Nbins))
    E2timeSum = np.zeros((len(Temporary_results), Nbins))
    E3timeSum = np.zeros((len(Temporary_results), Nbins))
    E4timeSum = np.zeros((len(Temporary_results), Nbins))
    m1E1timeSum = np.zeros((len(Temporary_results), Nbins))
    QttimeSum = np.zeros((len(Temporary_results), Nbins))


    NensembleExpected = 0
    for ResultsFile in ResultsFiles: 
        NensembleExpectedFile = ResultsFile['Nensemble']
        NensembleExpected += NensembleExpectedFile[...].astype(int)

    SystemLength = metadatas[0]['SystemLength'][...].astype(int)
    Temperature  = metadatas[0]['Temperature'][...].astype(float)
    subset_size = NensembleExpected // Nbins
    BinSize = np.zeros((Nbins))


    # Initialize counters
    # counter = 0
    Nensemble = 0
    Bin_counter = 0
    Bin_index = 0

    # Loop over each file
    for ResultsFile in ResultsFiles:
        EnsembleIndex = 0
        EnsembleIndexStr = str(EnsembleIndex)
        while ('/' + EnsembleIndexStr + '/TimeSeries/Equilibration/Magnetization') in ResultsFile:
            mtimeEnsemble = ResultsFile[('/' + EnsembleIndexStr + '/TimeSeries/Equilibration/Magnetization')]
            EtimeEnsemble = ResultsFile[('/' + EnsembleIndexStr + '/TimeSeries/Equilibration/Energy')]
            m1timeSum[: ,Bin_index] += mtimeEnsemble
            m2timeSum[: ,Bin_index] += np.square(mtimeEnsemble)
            m3timeSum[: ,Bin_index] += np.power(mtimeEnsemble, 3)
            m4timeSum[: ,Bin_index] += np.power(mtimeEnsemble, 4)
            E1timeSum[: ,Bin_index] += EtimeEnsemble
            E2timeSum[: ,Bin_index] += np.square(EtimeEnsemble)
            E3timeSum[: ,Bin_index] += np.power(EtimeEnsemble, 3)
            E4timeSum[: ,Bin_index] += np.power(EtimeEnsemble, 4)
            m1E1timeSum[: ,Bin_index] += np.array(mtimeEnsemble) * np.array(EtimeEnsemble)
            QttimeSum[: ,Bin_index] += mtimeEnsemble[0] * mtimeEnsemble
            Nensemble += 1
            # counter += 1
            EnsembleIndex += 1
            EnsembleIndexStr = str(EnsembleIndex)
            Bin_counter += 1
            if Bin_counter == subset_size:
                BinSize[Bin_index] = Bin_counter
                Bin_index += 1
                # Reset the counters
                Bin_counter = 0
                if Bin_index == Nbins:
                    break

    # If there are remaining values
    if Bin_counter > 0:
        BinSize[Bin_index] = Bin_counter
        # Bin_index += 1
        # Reset the counters
        Bin_counter = 0
        
    print(f"Nensemble = {Nensemble}")
    m1timeBins = m1timeSum / BinSize
    m2timeBins = m2timeSum / BinSize
    m3timeBins = m3timeSum / BinSize
    m4timeBins = m4timeSum / BinSize
    E1timeBins = E1timeSum / BinSize
    E2timeBins = E2timeSum / BinSize
    E3timeBins = E3timeSum / BinSize
    E4timeBins = E4timeSum / BinSize
    m1E1timeBins = m1E1timeSum / BinSize
    QtBins = QttimeSum / BinSize
    HeatCapacityBins = np.zeros((len(Temporary_results), Nbins))
    SusceptibilityBins = np.zeros((len(Temporary_results), Nbins))
    mDerBins = np.zeros((len(Temporary_results), Nbins))
    Binder2Bins = np.zeros((len(Temporary_results), Nbins))
    SystemSize= SystemLength ** 2
    N = SystemSize
    T = Temperature
    for i in range(m1timeBins.shape[1]):
        m1 = m1timeBins[:, i]
        m2 = m2timeBins[:, i]
        m3 = m3timeBins[:, i]
        m4 = m4timeBins[:, i]
        E1 = E1timeBins[:, i]
        E2 = E2timeBins[:, i]
        E3 = E3timeBins[:, i]
        E4 = E4timeBins[:, i]
        m1E1 = m1E1timeBins[:, i]
        HeatCapacityBins[:, i] = E2 / (E1 ** 2) - 1
        SusceptibilityBins[:, i] = m2 / (m1 ** 2) - 1
        mDerBins[:, i] = m1E1 / (m1 * E1) - 1
        Binder2Bins[:, i] = m2 / (m1 ** 2) - 1

    SaveFile= h5py.File(os.path.join(SubOutputPath, 'Results.h5'), 'w')
    for i in range(m1timeBins.shape[1]):
        SaveFile.create_dataset(f'/mtime1Bins/{i}', data=m1timeBins[:, i])
        SaveFile.create_dataset(f'/mtime2Bins/{i}', data=m2timeBins[:, i])
        SaveFile.create_dataset(f'/mtime3Bins/{i}', data=m3timeBins[:, i])
        SaveFile.create_dataset(f'/mtime4Bins/{i}', data=m4timeBins[:, i])
        SaveFile.create_dataset(f'/Etime1Bins/{i}', data=E1timeBins[:, i])
        SaveFile.create_dataset(f'/Etime2Bins/{i}', data=E2timeBins[:, i])
        SaveFile.create_dataset(f'/Etime3Bins/{i}', data=E3timeBins[:, i])
        SaveFile.create_dataset(f'/Etime4Bins/{i}', data=E4timeBins[:, i])
        SaveFile.create_dataset(f'/m1E1timeBins/{i}', data=m1E1timeBins[:, i])

    def getMeanErr(ParamBins):
        MeanParam= np.mean(ParamBins, axis=1)
        ErrorParam = np.std(ParamBins, axis=1)/ np.sqrt(ParamBins.shape[1])
        return MeanParam, ErrorParam

    MeanMagnetization, ErrorMagnetization  = getMeanErr(m1timeBins / N)
    MeanEnergy, ErrorEnergy  = getMeanErr(E1timeBins / N)
    MeanHeatCapacity, ErrorHeatCapacity  = getMeanErr(HeatCapacityBins)
    MeanSusceptibility, ErrorSusceptibility = getMeanErr(SusceptibilityBins)
    Mean_mDer, Error_mDer = getMeanErr(mDerBins)
    MeanBinder2, ErrorBinder2 = getMeanErr(Binder2Bins)
    MeanQt, ErrorQt = getMeanErr(QtBins)

    SaveFile.create_dataset('/Magnetization/mean', data=MeanMagnetization)
    SaveFile.create_dataset('/Magnetization/error', data=ErrorMagnetization)
    SaveFile.create_dataset('/Energy/mean', data=MeanEnergy)
    SaveFile.create_dataset('/Energy/error', data=ErrorEnergy)
    SaveFile.create_dataset('/HeatCapacity/mean', data=MeanHeatCapacity)
    SaveFile.create_dataset('/HeatCapacity/error', data=ErrorHeatCapacity)
    SaveFile.create_dataset('/Susceptibility/mean', data=MeanSusceptibility)
    SaveFile.create_dataset('/Susceptibility/error', data=ErrorSusceptibility)
    SaveFile.create_dataset('/mDer/mean', data=Mean_mDer)
    SaveFile.create_dataset('/mDer/error', data=Error_mDer)
    SaveFile.create_dataset('/Binder/mean', data=MeanBinder2)
    SaveFile.create_dataset('/Binder/error', data=ErrorBinder2)
    SaveFile.create_dataset('/Qt/mean', data=MeanQt)
    SaveFile.create_dataset('/Qt/error', data=ErrorQt)
    SaveFile.create_dataset('/Temperature', data=Temperature)
    SaveFile.create_dataset('/SystemLength', data=SystemLength)
    SaveFile.create_dataset('/SystemSize', data=SystemSize)
    SaveFile.create_dataset('/Nensemble', data=Nensemble)
    for ResultsFile in ResultsFiles:
        ResultsFile.close()
    for metadata in metadatas:
        metadata.close()
    SaveFile.close()

directory = 128\0.850000
..\..\..\copied\2024.09.02 10-13-18_NonEquilibrium_Copied\128\0.850000\1 exists
Nensemble = 6600
directory = 32\0.850000
..\..\..\copied\2024.09.02 10-13-18_NonEquilibrium_Copied\32\0.850000\1 exists
Nensemble = 6600
directory = 64\0.850000
..\..\..\copied\2024.09.02 10-13-18_NonEquilibrium_Copied\64\0.850000\1 exists
Nensemble = 6600


In [20]:
shutil.rmtree(CopyPath)